In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore

model = keras.models.load_model('plastics_model_v1.h5')

In [4]:
TRAIN_DIR = '/Users/michelangelozampieri/Desktop/plastics_CNN/seven_plastics'
TEST_DIR = '/Users/michelangelozampieri/Desktop/plastics_CNN/seven_plastics_test'
IMG_SIZE = (244, 244)
BATCH_SIZE = 32
NUM_CLASSES = 8
EPOCHS_FINE_TUNE = 10

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=40,  # Randomly rotate images by up to 40 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by 20%
    height_shift_range=0.2,  # Randomly shift images vertically by 20%
    shear_range=0.2,  # Randomly apply shearing transformations
    zoom_range=0.2,  # Randomly zoom in/out on images
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill in missing pixels after transformations
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    labels="inferred",  # Ensure labels are inferred from subdirectories
    label_mode="categorical",  # Use categorical labels for multi-class classification
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    labels="inferred",  # Ensure labels are inferred from subdirectories
    label_mode="categorical",  # Use categorical labels for validation
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 48 files belonging to 8 classes.
Found 294 files belonging to 8 classes.


In [7]:
model.trainable = True
for layer in model.layers[:-20]:  # Freeze all layers except the last 20
    layer.trainable = False

# Use a lower learning rate for fine-tuning
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

# Recompile model
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

# Train again with fine-tuning

history_fine = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=EPOCHS_FINE_TUNE
)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 23s 16s/step - accuracy: 0.9618 - loss: 0.2925 - val_accuracy: 0.3537 - val_loss: 1.7933
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.9271 - loss: 0.3061 - val_accuracy: 0.3537 - val_loss: 1.7906
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.9757 - loss: 0.2691 - val_accuracy: 0.3605 - val_loss: 1.7874
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 4s/step - accuracy: 0.9861 - loss: 0.2625 - val_accuracy: 0.3605 - val_loss: 1.7844
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 4s/step - accuracy: 1.0000 - loss: 0.2733 - val_accuracy: 0.3605 - val_loss: 1.7819
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 4s/step - accuracy: 1.0000 - loss: 0.2800 - val_accuracy: 0.3639 - val_loss: 1.7792
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 4s/step - accuracy: 0.9132 - loss: 0.3645 - val_accuracy: 0.3639 - val_loss: 1.7761
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 4s/step - accuracy: 0.9618 - loss: 0.2624 - val_accuracy: 0.3605 - val_loss: 1.7733
Epoch 9/10
2/2

In [9]:
model.save('plastics_model_v2.keras')

In [10]:
val_loss, val_accuracy = model.evaluate(test_dataset)
print(f"Validation accuracy: {val_accuracy}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 450ms/step - accuracy: 0.4084 - loss: 1.7465
Validation accuracy: 0.37755101919174194
